## ファイルのインポート 

In [2]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

In [3]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time


## 各ウェブサイトにURLをテーブルから取得

In [99]:
#過去の試合一覧が乗っているHPを取得
url = "https://www.football-lab.jp/nago/match/?year=2019"
res = requests.get(url)

In [5]:
#beautifulsoup(解析対象のHTML/XML, 利用するパーサー)
soup = BeautifulSoup(res.text, "html.parser")


In [12]:
#home のみ取り出す
home_aways = soup.find_all('span', class_='home')
home_url=[]
for i in home_aways:
#    home_away = soup.find('span', class_='away') #awayを見つけ出す
    Parent = i.parent.parent #親をたどる
    elems = Parent.find('a').next_element.next_element.next_element.find('a')['href'] #子要素を取り出す。
    home_url.append(elems)
home_url

['/nago/report/?year=2019&month=03&date=02',
 '/nago/report/?year=2019&month=03&date=30',
 '/nago/report/?year=2019&month=04&date=20',
 '/nago/report/?year=2019&month=04&date=28',
 '/nago/report/?year=2019&month=05&date=12',
 '/nago/report/?year=2019&month=05&date=26',
 '/nago/report/?year=2019&month=06&date=22',
 '/nago/report/?year=2019&month=07&date=07',
 '/nago/report/?year=2019&month=07&date=20',
 '/nago/report/?year=2019&month=08&date=10',
 '/nago/report/?year=2019&month=08&date=24',
 '/nago/report/?year=2019&month=08&date=30',
 '/nago/report/?year=2019&month=10&date=05',
 '/nago/report/?year=2019&month=10&date=19',
 '/nago/report/?year=2019&month=11&date=09',
 '/nago/report/?year=2019&month=11&date=23',
 '/nago/report/?year=2019&month=12&date=07']

In [197]:
#リスト型にする
# pickup_links=[elem.attrs["href"] for elem in elems]
home_url=['https://www.football-lab.jp'+home_url[i] for i in range(len(home_url))]
home_url


['https://www.football-lab.jp/nago/report/?year=2019&month=03&date=02',
 'https://www.football-lab.jp/nago/report/?year=2019&month=03&date=30',
 'https://www.football-lab.jp/nago/report/?year=2019&month=04&date=20',
 'https://www.football-lab.jp/nago/report/?year=2019&month=04&date=28',
 'https://www.football-lab.jp/nago/report/?year=2019&month=05&date=12',
 'https://www.football-lab.jp/nago/report/?year=2019&month=05&date=26',
 'https://www.football-lab.jp/nago/report/?year=2019&month=06&date=22',
 'https://www.football-lab.jp/nago/report/?year=2019&month=07&date=07',
 'https://www.football-lab.jp/nago/report/?year=2019&month=07&date=20',
 'https://www.football-lab.jp/nago/report/?year=2019&month=08&date=10',
 'https://www.football-lab.jp/nago/report/?year=2019&month=08&date=24',
 'https://www.football-lab.jp/nago/report/?year=2019&month=08&date=30',
 'https://www.football-lab.jp/nago/report/?year=2019&month=10&date=05',
 'https://www.football-lab.jp/nago/report/?year=2019&month=10&da

## `ウェブサイト立ち上げ

In [101]:
# ブラウザ起動
PATH = "C:\Program Files (x86)\chromedriver.exe"
# options = Options()
#options.add_argument("--incognito") # プライベートブラウズモードで開く

driver = webdriver.Chrome(executable_path = PATH)

# windows size設定(80%表示で最適な window size)
driver.set_window_size(840,1000)
#Webドライバーのタイムアウト時間を10秒に設定
wait = WebDriverWait(driver, 10)

## テーブル取得

In [97]:
def get_table(urls):
    dfs=[]
    for url in urls: #一旦３つくらい
        driver.get(url)
    # CLASS名指定したページ上の要素が読み込まれるまで待機（15秒でタイムアウト判定）
        WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CLASS_NAME, 'statsTbl6 ')))
        elem_table = driver.find_element_by_xpath("/html/body/article/div[10]/table[2]")
        time.sleep(3)
        html = elem_table.get_attribute('outerHTML')
        df = pd.read_html(html) #ここではリスト

        #対戦チームの
        elem_home = driver.find_element_by_css_selector('#nago > article > div:nth-child(10) > div:nth-child(6) > em:nth-child(1)')
        elem_home_team = driver.find_element_by_css_selector('#nago > article > div:nth-child(10) > div:nth-child(6) > em:nth-child(2) > span.dsktp')
        elem_away = driver.find_element_by_css_selector('#nago > article > div:nth-child(10) > div:nth-child(6) > em:nth-child(4)')
        elem_away_team = driver.find_element_by_css_selector('#nago > article > div:nth-child(10) > div:nth-child(6) > em:nth-child(5) > span.dsktp')

        dfs.append([df,elem_home.text,elem_home_team.text,elem_away.text,elem_away_team.text])
    return dfs
    # teams_home_away.append([elem_home.text,elem_home_team.text,elem_away.text,elem_away_team.text])

In [200]:
home_dfs = get_table(home_url)
len(home_dfs)

17

In [201]:
home_dfs[0][0][0]

,今季平均,成功率,総数,Unnamed: 3,総数.1,成功率.1,今季平均.1
0,ゴール期待値,ゴール期待値,ゴール期待値,ゴール期待値,NaN,NaN,NaN
1,1.583,-,1.17,ゴール期待値,0.423,-,1.507
2,シュート,シュート,シュート,シュート,NaN,NaN,NaN
3,14.2,(13.3%),15,シュート,4,(0.0%),13.300
4,枠内シュート,枠内シュート,枠内シュート,枠内シュート,NaN,NaN,NaN
5,4.5,-,4,枠内シュート,2,-,4.600
6,PKによるシュート,PKによるシュート,PKによるシュート,PKによるシュート,NaN,NaN,NaN
7,0.0,-,0,PKによるシュート,0,-,0.000
8,パス,パス,パス,パス,NaN,NaN,NaN
9,609.0,(87.7%),651,パス,403,(81.9%),480.000


In [108]:
def change_column_name(dfs):
    #列名をそれぞれ変える
    change_dfs=[]
    for i in range(len(dfs)):
        stats_df = pd.DataFrame(dfs[i][0][0])
            # カラム名変更
        columns_name = [dfs[i][1]+"_"+dfs[i][2]+"_2019平均",
                            dfs[i][1]+"_"+dfs[i][2]+"_成功率",
                            dfs[i][1]+"_"+dfs[i][2]+"_総数",
                            "スタッツ",
                            dfs[i][3]+"_"+dfs[i][4]+"_総数",
                            dfs[i][3]+"_"+dfs[i][4]+"_成功率",
                            dfs[i][3]+"_"+dfs[i][4]+"_2019平均"]
        stats_df.columns = columns_name
        change_dfs.append(stats_df)
        # stats_df.dropna(axis = 0, how = 'any', inplace=True) #一つでもNanが行に含まれてたら削除
        # stats_df.set_index("スタッツ", inplace=True, drop=True) #Dropでsetindexの列を残さない
    return change_dfs

In [203]:
home_dfs = change_column_name(home_dfs)

In [204]:

len(home_dfs)

17

In [107]:
def organize_df(dfs):
    organize_df1=[]
    for i in range(len(dfs)):
        # stats_df = pd.DataFrame(dfs[i][0][0])

        #データフレームを整理する
        dfs[i].dropna(axis = 0, how = 'any', inplace=True) #一つでもNanが行に含まれてたら削除
        dfs[i].set_index("スタッツ", inplace=True, drop=True) #Dropでsetindexの列を残さない

        #置換
        dfs[i].iloc[:,2] = dfs[i].iloc[:,2].str.replace("m", "")
        dfs[i].iloc[:,2] = dfs[i].iloc[:,2].str.replace("%", "")
        dfs[i].iloc[:,2] = dfs[i].iloc[:,2].str.replace(",", "")
        dfs[i].iloc[:,3] = dfs[i].iloc[:,3].str.replace("m", "")
        dfs[i].iloc[:,3] = dfs[i].iloc[:,3].str.replace("%", "")
        dfs[i].iloc[:,3] = dfs[i].iloc[:,3].str.replace(",", "")
        dfs[i].iloc[:,1] = dfs[i].iloc[:,1].str.replace("%", "")
        dfs[i].iloc[:,1] = dfs[i].iloc[:,1].str.replace(")", "")
        dfs[i].iloc[:,1] = dfs[i].iloc[:,1].str.replace("(", "")
        dfs[i].iloc[:,1] = dfs[i].iloc[:,1].str.replace("-", "0")
        dfs[i].iloc[:,4] = dfs[i].iloc[:,4].str.replace("%", "")
        dfs[i].iloc[:,4] = dfs[i].iloc[:,4].str.replace(")", "")
        dfs[i].iloc[:,4] = dfs[i].iloc[:,4].str.replace("(", "")
        dfs[i].iloc[:,4] = dfs[i].iloc[:,4].str.replace("-", "0")
        
                #float型にする
        dfs[i].iloc[:,0] = dfs[i].iloc[:,0].astype(float)
        dfs[i].iloc[:,1] = dfs[i].iloc[:,1].astype(float)
        dfs[i].iloc[:,2] = dfs[i].iloc[:,2].astype(float)
        dfs[i].iloc[:,3] = dfs[i].iloc[:,3].astype(float)
        dfs[i].iloc[:,4] = dfs[i].iloc[:,4].astype(float)
        dfs[i].iloc[:,5] = dfs[i].iloc[:,5].astype(float)


        organize_df1.append(dfs[i])

    # return organize_df

In [206]:
organize_df(home_dfs)

In [208]:
home_dfs[16]

,HOME_名古屋グランパス_2019平均,HOME_名古屋グランパス_成功率,HOME_名古屋グランパス_総数,AWAY_鹿島アントラーズ_総数,AWAY_鹿島アントラーズ_成功率,AWAY_鹿島アントラーズ_2019平均
スタッツ,,,,,,
ゴール期待値,1.583,0.0,1.091,1.594,0.0,1.485
シュート,14.200,0.0,8.000,10.000,0.0,12.400
枠内シュート,4.500,0.0,1.000,2.000,0.0,4.300
PKによるシュート,0.000,0.0,0.000,0.000,0.0,0.100
パス,609.000,85.4,680.000,430.000,79.5,438.200
クロス,14.800,14.3,14.000,15.000,33.3,14.400
直接ＦＫ,13.600,0.0,13.000,11.000,0.0,12.600
間接ＦＫ,2.600,0.0,2.000,2.000,0.0,1.900
ＣＫ,5.600,0.0,4.000,4.000,0.0,4.800


## awaygameのデータ取得

In [183]:
#home のみ取り出す
home_aways = soup.find_all('span', class_='away')
away_url=[]
for i in home_aways:
#    home_away = soup.find('span', class_='away') #awayを見つけ出す
    Parent = i.parent.parent #親をたどる
    elems = Parent.find('a').next_element.next_element.next_element.find('a')['href'] #子要素を取り出す。
    away_url.append(elems)
home_url
#リスト型にする
# pickup_links=[elem.attrs["href"] for elem in elems]
away_url=['https://www.football-lab.jp'+away_url[i] for i in range(len(away_url))]
away_url


['https://www.football-lab.jp/nago/report/?year=2019&month=02&date=23',
 'https://www.football-lab.jp/nago/report/?year=2019&month=03&date=09',
 'https://www.football-lab.jp/nago/report/?year=2019&month=03&date=17',
 'https://www.football-lab.jp/nago/report/?year=2019&month=04&date=05',
 'https://www.football-lab.jp/nago/report/?year=2019&month=04&date=13',
 'https://www.football-lab.jp/nago/report/?year=2019&month=05&date=04',
 'https://www.football-lab.jp/nago/report/?year=2019&month=05&date=17',
 'https://www.football-lab.jp/nago/report/?year=2019&month=06&date=01',
 'https://www.football-lab.jp/nago/report/?year=2019&month=06&date=15',
 'https://www.football-lab.jp/nago/report/?year=2019&month=06&date=30',
 'https://www.football-lab.jp/nago/report/?year=2019&month=07&date=13',
 'https://www.football-lab.jp/nago/report/?year=2019&month=08&date=04',
 'https://www.football-lab.jp/nago/report/?year=2019&month=08&date=18',
 'https://www.football-lab.jp/nago/report/?year=2019&month=09&da

In [184]:
away_dfs = get_table(away_url)

In [185]:
away_dfs = change_column_name(away_dfs)

In [186]:
organize_df(away_dfs)

In [187]:
away_dfs[1]

,HOME_ガンバ大阪_2019平均,HOME_ガンバ大阪_成功率,HOME_ガンバ大阪_総数,AWAY_名古屋グランパス_総数,AWAY_名古屋グランパス_成功率,AWAY_名古屋グランパス_2019平均
スタッツ,,,,,,
ゴール期待値,1.433,0.0,1.576,1.462,0.0,1.583
シュート,14.600,10.0,10.000,17.000,11.8,14.200
枠内シュート,4.900,0.0,6.000,9.000,0.0,4.500
PKによるシュート,0.100,0.0,1.000,0.000,0.0,0.000
パス,587.000,76.8,479.000,514.000,78.2,609.000
クロス,15.100,12.5,8.000,8.000,37.5,14.800
直接ＦＫ,11.000,0.0,15.000,16.000,0.0,13.600
間接ＦＫ,1.400,0.0,1.000,7.000,0.0,2.600
ＣＫ,5.500,0.0,3.000,7.000,0.0,5.600


In [188]:
dfs[1]

[[          今季平均         成功率          総数  Unnamed: 3      総数.1    成功率.1  \
  0       ゴール期待値      ゴール期待値      ゴール期待値      ゴール期待値       NaN      NaN   
  1        1.296           -       0.899      ゴール期待値     1.411        -   
  2         シュート        シュート        シュート        シュート       NaN      NaN   
  3         11.5      (7.7%)          13        シュート        14   (0.0%)   
  4       枠内シュート      枠内シュート      枠内シュート      枠内シュート       NaN      NaN   
  5          3.9           -           4      枠内シュート         6        -   
  6    PKによるシュート   PKによるシュート   PKによるシュート   PKによるシュート       NaN      NaN   
  7          0.1           -           0   PKによるシュート         0        -   
  8           パス          パス          パス          パス       NaN      NaN   
  9        412.5     (65.4%)         283          パス       606  (80.7%)   
  10         クロス         クロス         クロス         クロス       NaN      NaN   
  11        13.8     (12.5%)           8         クロス        18  (27.8%)   
  12        直接ＦＫ        直

In [94]:
#スタジアム毎のurlを取得する できた
mizuho_url=[]
toyota_url=[]
for tr in soup.select('table.statsTbl10 tbody tr'):
    tds = tr.select('td')
    if tds[6].text=="豊田ス":
        toyota_url.append(tds[4].find("a")['href'] )

    elif tds[6].text=="パロ瑞穂":
        mizuho_url.append(tds[4].find("a")['href'] )

toyota_url=['https://www.football-lab.jp'+toyota_url[i] for i in range(len(toyota_url))]
mizuho_url=['https://www.football-lab.jp'+mizuho_url[i] for i in range(len(mizuho_url))]

In [132]:
# toyota_dfs = get_table(toyota_url)
mizuho_dfs = get_table(mizuho_url)

In [133]:
mizuho_dfs

[[[          今季平均         成功率          総数  Unnamed: 3      総数.1    成功率.1  \
   0       ゴール期待値      ゴール期待値      ゴール期待値      ゴール期待値       NaN      NaN   
   1        1.583           -        1.17      ゴール期待値     0.423        -   
   2         シュート        シュート        シュート        シュート       NaN      NaN   
   3         14.2     (13.3%)          15        シュート         4   (0.0%)   
   4       枠内シュート      枠内シュート      枠内シュート      枠内シュート       NaN      NaN   
   5          4.5           -           4      枠内シュート         2        -   
   6    PKによるシュート   PKによるシュート   PKによるシュート   PKによるシュート       NaN      NaN   
   7          0.0           -           0   PKによるシュート         0        -   
   8           パス          パス          パス          パス       NaN      NaN   
   9        609.0     (87.7%)         651          パス       403  (81.9%)   
   10         クロス         クロス         クロス         クロス       NaN      NaN   
   11        14.8     (11.8%)          17         クロス         6  (16.7%)   
   12       

In [134]:
# toyota_dfs = change_column_name(toyota_dfs)
mizuho_dfs = change_column_name(mizuho_dfs)

In [137]:
mizuho_dfs[1]

,HOME_名古屋グランパス_2019平均,HOME_名古屋グランパス_成功率,HOME_名古屋グランパス_総数,スタッツ,AWAY_清水エスパルス_総数,AWAY_清水エスパルス_成功率,AWAY_清水エスパルス_2019平均
0,ゴール期待値,ゴール期待値,ゴール期待値,ゴール期待値,NaN,NaN,NaN
1,1.583,-,1.304,ゴール期待値,1.434,-,1.313
2,シュート,シュート,シュート,シュート,NaN,NaN,NaN
3,14.2,(8.3%),12,シュート,12,(16.7%),12.600
4,枠内シュート,枠内シュート,枠内シュート,枠内シュート,NaN,NaN,NaN
5,4.5,-,3,枠内シュート,6,-,4.200
6,PKによるシュート,PKによるシュート,PKによるシュート,PKによるシュート,NaN,NaN,NaN
7,0.0,-,0,PKによるシュート,0,-,0.100
8,パス,パス,パス,パス,NaN,NaN,NaN
9,609.0,(81.9%),728,パス,264,(61.0%),393.300


In [138]:
# toyota_dfs = organize_df(toyota_dfs)
organize_df(mizuho_dfs)

In [141]:
mizuho_dfs[1]

,HOME_名古屋グランパス_2019平均,HOME_名古屋グランパス_成功率,HOME_名古屋グランパス_総数,AWAY_清水エスパルス_総数,AWAY_清水エスパルス_成功率,AWAY_清水エスパルス_2019平均
スタッツ,,,,,,
ゴール期待値,1.583,0.0,1.304,1.434,0.0,1.313
シュート,14.200,8.3,12.000,12.000,16.7,12.600
枠内シュート,4.500,0.0,3.000,6.000,0.0,4.200
PKによるシュート,0.000,0.0,0.000,0.000,0.0,0.100
パス,609.000,81.9,728.000,264.000,61.0,393.300
クロス,14.800,15.0,20.000,6.000,83.3,13.500
直接ＦＫ,13.600,0.0,16.000,15.000,0.0,11.100
間接ＦＫ,2.600,0.0,4.000,0.000,0.0,1.600
ＣＫ,5.600,0.0,3.000,3.000,0.0,4.200
